### Use Case 15

Description: Demonstrate in the event of a data leak how we can audit access and activity within the system.

### **Overview**
This worksheet demonstrates **Snowflake's logging for whenever a user accesses data**, enabling **audit capabilities for data accessed and queries executed**.

Snowflake's built-in [ACCOUNT_USAGE](https://docs.snowflake.com/en/sql-reference/account-usage) schema provides 100+ views and table functions out of the box that provide a variety of insights for Snowflake administrators.

### Login History

The [LOGIN_HISTORY](https://docs.snowflake.com/en/sql-reference/account-usage/login_history) view provides logs for all login attempts, successful and unsuccessful.

In [ ]:
use database snowflake;
use schema account_usage;

select 
    *
from login_history
order by event_timestamp desc
limit 1000;

### Query History

The [QUERY_HISTORY](https://docs.snowflake.com/en/sql-reference/functions/query_history) view can be used to query Snowflake query history by various dimensions (time range, session, user, warehouse, etc.).

In addition to using QUERY_HISTORY, you can use the Snowsight UI to dig deeper into the Query Profile for any individual query. This view provides a graphical representation with performance metrics to help **identify performance bottlenecks** within a specific query.

In [ ]:
use database snowflake;
use schema account_usage;

select
    *
from query_history
order by end_time desc
limit 100;

### Access History

The [ACCESS_HISTORY](https://docs.snowflake.com/en/sql-reference/account-usage/access_history) view can be used to **audit when any user reads data and/or performs a data write operation**, such as INSERT, UPDATE, and DELETE along with variations of the COPY command. The view also provides the capability to audit **lineage from the source data object to the target data object**.

In [ ]:
SELECT user_name
       , query_id
       , query_start_time
       , direct_objects_accessed
       , base_objects_accessed
FROM access_history
ORDER BY 1, 3 desc
LIMIT 100;

In [ ]:
SELECT distinct user_name
FROM access_history
     , lateral flatten(base_objects_accessed) f1
WHERE f1.value:"objectId"::int=50211
AND f1.value:"objectDomain"::string='Table'
AND query_start_time >= dateadd('day', -30, current_timestamp())
;

In [ ]:
SELECT query_id
       , query_start_time
FROM access_history
     , lateral flatten(base_objects_accessed) f1
WHERE f1.value:"objectId"::int=50211
AND f1.value:"objectDomain"::string='Table'
AND query_start_time >= dateadd('day', -30, current_timestamp())
;

In [ ]:
SELECT distinct f4.value AS column_name
FROM access_history
     , lateral flatten(base_objects_accessed) f1
     , lateral flatten(f1.value) f2
     , lateral flatten(f2.value) f3
     , lateral flatten(f3.value) f4
WHERE f1.value:"objectId"::int=50211
AND f1.value:"objectDomain"::string='Table'
AND f4.key='columnName'
;

In [ ]:
// 1

select
  directSources.value: "objectId" as source_object_id,
  directSources.value: "objectName" as source_object_name,
  directSources.value: "columnName" as source_column_name,
  'DIRECT' as source_column_type,
  om.value: "objectName" as target_object_name,
  columns_modified.value: "columnName" as target_column_name
from
  (
    select
      *
    from
      snowflake.account_usage.access_history
  ) t,
  lateral flatten(input => t.OBJECTS_MODIFIED) om,
  lateral flatten(input => om.value: "columns", outer => true) columns_modified,
  lateral flatten(
    input => columns_modified.value: "directSources",
    outer => true
  ) directSources

union

// 2

select
  baseSources.value: "objectId" as source_object_id,
  baseSources.value: "objectName" as source_object_name,
  baseSources.value: "columnName" as source_column_name,
  'BASE' as source_column_type,
  om.value: "objectName" as target_object_name,
  columns_modified.value: "columnName" as target_column_name
from
  (
    select
      *
    from
      snowflake.account_usage.access_history
  ) t,
  lateral flatten(input => t.OBJECTS_MODIFIED) om,
  lateral flatten(input => om.value: "columns", outer => true) columns_modified,
  lateral flatten(
    input => columns_modified.value: "baseSources",
    outer => true
  ) baseSources
;